In [1]:
#add name of folder in data which contains the sar data at end of list
datelist = ["January-10-2023", "January-22-2023", "February-03-2023", "February-15-2023", "February-27-2023", "March-11-2023", "March-23-2023", "April-04-2023", "April-16-2023", "April-28-2023", "May-10-2023", "May-22-2023", "June-03-2023", "June-15-2023"]

# datelist = ["April-28-2023", "May-10-2023", "May-22-2023"]

In [2]:
import json
import pandas as pd
import geopandas as gpd
import shapely
from shapely import geometry
import matplotlib.pyplot as plt
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from jarvis.aqua import aeratorDetection as ad
# from jarvis.aqua import detectWaterInAPond
from statistics import mode

In [ ]:
features = []

In [ ]:
with open('./fetched.json', 'r') as f:
  data = json.load(f)

In [ ]:
data = data["data"]["pond_blue"]

In [ ]:
data[0]

In [ ]:
def detectWaterInAPond(dbVVBand, thresholdValue=-17):
    """
    For input sar dB VV band, detect whether the image has water present or not
    -----------------------------------------
    Input arguments:
    dbVVBand: a numpy array of shape (M,N) containing the dB VV band from a Sentinel 1 image
    thresholdValue: threshold to determine if body has water or not
    -----------------------------------------
    Output:
    water: an integer value where 0 means no water and 1 means water
    """
    minValue = 100
    values = []
    for i in range(0, dbVVBand.shape[0]):
        for j in range(0, dbVVBand.shape[1]):
            if dbVVBand[i][j]==-999:
                continue
            else:
                if dbVVBand[i][j] < minValue:
                    minValue = dbVVBand[i][j]
                    window = dbVVBand[i-1:j+1, i-1:j+1]
                    window = window.flatten()
                    for w in window:
                        if w==-999:
                            continue
                        else:
                            values.append(w)
    if len(values)>0:
        averageValue = sum(values)/len(values)
    else:
        averageValue = 0
    if averageValue<-17:
        water = 1
    else:
        water = 0
    return averageValue

In [ ]:
for d in tqdm(data):
    sampleData = {"type": "Feature",
              "properties": {},
              "geometry": {
                  "type": "Polygon"
              }}
    sampleData["properties"]["id"] = d["id"]
    sampleData["properties"]["name"] = d["name"]
    sampleData["properties"]["farmer_name"] = d["farm_blue"]["farmer_name"]
    sampleData["properties"]["feed_supplier"] = d["farm_blue"]["feed_supplier"]
    sampleData["properties"]["village_name"] = d["farm_blue"]["village_name"]
    sampleData["geometry"]["coordinates"] = d["polygon"]["coordinates"]
    polygon = geometry.Polygon(d['polygon']['coordinates'][0])
    with rasterio.open(os.path.join("./data/May-22-2023/dB_vv.tif")) as src:
        croppedImage, _ = mask(src, [polygon], crop = True, nodata=-999)
        croppedImage = croppedImage[0]
        waterstatus = detectWaterInAPond(croppedImage, thresholdValue=-10)
        sampleData["properties"]["water"] = waterstatus
    features.append(sampleData)

In [ ]:
features[0]

In [ ]:
geodata = {"type": "FeatureCollection",
           "features": features}

In [ ]:
with open(os.path.join("./may22.geojson"), 'w+') as f:
    json.dump(geodata, f, indent=2)

### Creating geojson with properties for each date
Format of geojsons:
- each date will have it's respective geojson
- there will be a master geojson containing the following details:
  - current latest status of each pond
  - time series detail for each pond, where each time stamp will have data regarding water and aerator in pond

In [3]:
def detectWaterInAPond(dbVVBand, thresholdValue=-17):
    """
    For input sar dB VV band, detect whether the image has water present or not
    -----------------------------------------
    Input arguments:
    dbVVBand: a numpy array of shape (M,N) containing the dB VV band from a Sentinel 1 image
    thresholdValue: threshold to determine if body has water or not
    -----------------------------------------
    Output:
    water: an integer value where 0 means no water and 1 means water
    """
    minValue = 100
    values = []
    for i in range(0, dbVVBand.shape[0]):
        for j in range(0, dbVVBand.shape[1]):
            if dbVVBand[i][j]==-999:
                continue
            else:
                if dbVVBand[i][j] < minValue:
                    minValue = dbVVBand[i][j]
                    window = dbVVBand[i-1:j+1, i-1:j+1]
                    window = window.flatten()
                    for w in window:
                        if w==-999:
                            continue
                        else:
                            values.append(w)
    if len(values)>0:
        averageValue = sum(values)/len(values)
    else:
        averageValue = 0
    if averageValue<-15:
        water = 1
    else:
        water = 0
    return water

In [4]:
for date in datelist:
    with open('./fetched.json', 'r') as f:
        data = json.load(f)
    data = data["data"]["pond_blue"]
    if os.path.exists(os.path.join("./data",date)):
        print("Doing analysis for:",date)
        os.makedirs(os.path.join("./output",date))
        for i in tqdm(range(0, len(data))):
            polygon = geometry.Polygon(data[i]['polygon']['coordinates'][0])
            # farmdid = data[i]['farm_id']
            id = data[i]['id']
            name = data[i]['name']
            farmer_name = data[i]['farm_blue']['farmer_name']
            feed_supplier = data[i]['farm_blue']['feed_supplier']
            village_name = data[i]['farm_blue']['village_name']
            with rasterio.open(os.path.join("./data",date,"dB_vv.tif")) as src:
                croppedImage, _ = mask(src, [polygon], crop = True, nodata=-999)
                croppedImage = croppedImage[0]
                waterstatus = detectWaterInAPond(croppedImage, thresholdValue=-10)
                if waterstatus:
                    with rasterio.open(os.path.join("./data",date,"linear_vv.tif")) as src:
                        croppedImage, _ = rasterio.mask.mask(src, [polygon], crop = True)
                        croppedImage = croppedImage[0]
                        aeratorstatus = ad.detectAerators(croppedImage)
                        if aeratorstatus:
                            aeratorstatus = 1
                        else:
                            aeratorstatus = 0
                else:
                    aeratorstatus = 0
            data[i]["properties"] = {"Water": waterstatus,
                                     "Aerator": aeratorstatus,
                                    #  "farm_id": farmdid,
                                     "id": id,
                                     "name": name,
                                     "farmer_name": farmer_name,
                                     "feed_supplier": feed_supplier,
                                     "village_name": village_name}
        with open(os.path.join("./output",date,"properties.geojson"), 'w+') as f:
            json.dump(data, f)
    else:
        print("data for date does not exist, skipping")

Doing analysis for: January-10-2023


  0%|          | 0/7788 [00:00<?, ?it/s]/Users/azhanmohammed/opt/anaconda3/lib/python3.9/site-packages/jarvis/aqua/aeratorDetection.py:45: RuntimeWarning: invalid value encountered in scalar divide
  if ((center_pixel-mean)/std) > threshold:
100%|██████████| 7788/7788 [00:29<00:00, 264.75it/s]


Doing analysis for: January-22-2023


100%|██████████| 7788/7788 [00:29<00:00, 266.36it/s]


Doing analysis for: February-03-2023


...

100%|██████████| 7788/7788 [00:34<00:00, 227.67it/s]


Doing analysis for: June-03-2023


100%|██████████| 7788/7788 [00:37<00:00, 210.19it/s]


Doing analysis for: June-15-2023


100%|██████████| 7788/7788 [00:35<00:00, 218.90it/s]


### time series logic to create mastergeojson

In [5]:
#read the data from genreated geojson

datajan10 = pd.read_json("./output/January-10-2023/properties.geojson")
datajan22 = pd.read_json("./output/January-22-2023/properties.geojson")                        
datafeb3 = pd.read_json("./output/February-03-2023/properties.geojson")
datafeb15 = pd.read_json("./output/February-15-2023/properties.geojson")
datafeb27 = pd.read_json("./output/February-27-2023/properties.geojson")   
datamarch11 = pd.read_json("./output/March-11-2023/properties.geojson")
datamarch23 = pd.read_json("./output/March-23-2023/properties.geojson")
dataapril4 = pd.read_json("./output/April-04-2023/properties.geojson")
dataapril16 = pd.read_json("./output/April-16-2023/properties.geojson")
dataapril28 = pd.read_json("./output/April-28-2023/properties.geojson")
datamay10 = pd.read_json("./output/May-10-2023/properties.geojson")
datamay22 = pd.read_json("./output/May-22-2023/properties.geojson")
datajune03 = pd.read_json("./output/June-03-2023/properties.geojson")
datajune15 = pd.read_json("./output/June-15-2023/properties.geojson")

In [6]:
#add name of folder in data which contains the sar data at end of list

datelist = ["January-10-2023", "January-22-2023", "February-03-2023", "February-15-2023", "February-27-2023", "March-11-2023", "March-23-2023", "April-04-2023", "April-16-2023", "April-28-2023", "May-10-2023", "May-22-2023", "June-03-2023", "June-15-2023"]

In [10]:
#add name of folder as dictionary key, and data as dictionary value
datacollection = {"January-10-2023": datajan10,
                  "January-22-2023": datajan22,
                  "February-03-2023": datafeb3,
                  "February-15-2023": datafeb15,
                  "February-27-2023": datafeb27,
                  "March-11-2023": datamarch11,
                  "March-23-2023": datamarch23,
                  "April-04-2023": dataapril4,
                  "April-16-2023": dataapril16,
                  "April-28-2023": dataapril28,
                  "May-10-2023": datamay10,
                  "May-22-2023": datamay22,
                  "June-03-2023": datajune03,
                  "June-15-2023": datajune15}

#add name of folder as dictionary key, and data as dictionary value
# datacollection = {"April-28-2023": dataapril28,
#                   "May-10-2023": datamay10,
#                   "May-22-2023": datamay22}

In [11]:
with open('./fetched.json', 'r') as f:
  data = json.load(f)
data = data["data"]["pond_blue"]

In [12]:
for i in range(0, len(data)):
    id = data[i]['id']
    name = data[i]['name']
    farmer_name = data[i]['farm_blue']['farmer_name']
    feed_supplier = data[i]['farm_blue']['feed_supplier']
    village_name = data[i]['farm_blue']['village_name']
    timeSeries = []
    for date in datelist:
        currentTime = {}
        datedata = datacollection[date]
        currentTime["time"] = date
        currentTime["Water"] = datedata["properties"][i]["Water"]
        currentTime["Aerator"] = datedata["properties"][i]["Aerator"]
        timeSeries.append(currentTime)
    data[i]["properties"] = {"farmer_name": farmer_name,
                             "feed_supplier": feed_supplier,
                             "village_name": village_name,
                             "id": id,
                             "name": name,
                             "time_series": timeSeries}

In [13]:
unused = 0
active = 0
underpumping = 0
harvested = 0
for i in range(0, len(data)):
    for j in range(1, len((data[i]["properties"]["time_series"]))):
        aeratorNow = data[i]["properties"]["time_series"][j]["Aerator"]
        waterNow = data[i]["properties"]["time_series"][j]["Water"]
        aeratorEarlier = data[i]["properties"]["time_series"][j-1]["Aerator"]
        waterEarlier = data[i]["properties"]["time_series"][j-1]["Water"]
        if waterEarlier==0 and waterNow==0:
            status = "empty"
        elif waterEarlier==0 and waterNow==1:
            if aeratorNow==1:
                status = "aerated"
            else:
                status = "pumped"
        elif waterEarlier==1 and waterNow==1:
            if aeratorEarlier==0 and aeratorNow==1:
                status = "aerated"
            elif aeratorEarlier==0 and aeratorNow==0:
                status = "pumped"
            elif aeratorEarlier==1 and aeratorNow==1:
                status = "aerated"
            elif aeratorEarlier==1 and aeratorNow==0:
                status = "harvested"
        elif waterEarlier==1 and waterNow==0:
            if aeratorEarlier==1:
                status = "harvested"
            else:
                status = "empty"
        data[i]["properties"]["time_series"][j]["status"] = status
        if j == len((data[i]["properties"]["time_series"]))-1:
            data[i]["properties"]["farm_status"] = status
            if status=="aerated":
                active = active + 1
            if status=="empty":
                unused = unused + 1
            if status=="pumped":
                underpumping = underpumping + 1
            if status=="harvested":
                harvested = harvested + 1

In [14]:
for i in range(0, len(data)):
    currentStatus = data[i]["properties"]["time_series"][1]["status"]
    statusList = [data[i]["properties"]["time_series"][1]["status"]]
    timeList = [data[i]["properties"]["time_series"][1]["time"]]
    for k in range(2, len(data[i]["properties"]["time_series"])):
        if currentStatus=="empty":
            if data[i]["properties"]["time_series"][k]["status"]==currentStatus:
                timeList.pop(-1)
                statusList.pop(-1)
                timeList.append(data[i]["properties"]["time_series"][k]["time"])
                statusList.append(data[i]["properties"]["time_series"][k]["status"])
            elif data[i]["properties"]["time_series"][k]["status"]=="aerated":
                currentStatus = data[i]["properties"]["time_series"][k]["status"]
                statusList.append("pumped and aerated")
                timeList.append(data[i]["properties"]["time_series"][k]["time"])
            elif data[i]["properties"]["time_series"][k]["status"]=="pumped":
                currentStatus = data[i]["properties"]["time_series"][k]["status"]
                statusList.append(data[i]["properties"]["time_series"][k]["status"])
                timeList.append(data[i]["properties"]["time_series"][k]["time"])
        elif currentStatus=="pumped":
            if data[i]["properties"]["time_series"][k]["status"]=="aerated":
                currentStatus = data[i]["properties"]["time_series"][k]["status"]
                statusList.append(data[i]["properties"]["time_series"][k]["status"])
                timeList.append(data[i]["properties"]["time_series"][k]["time"])
            elif data[i]["properties"]["time_series"][k]["status"]=="harvested":
                currentStatus = data[i]["properties"]["time_series"][k]["status"]
                statusList.append(data[i]["properties"]["time_series"][k]["status"])
                timeList.append(data[i]["properties"]["time_series"][k]["time"])
        elif currentStatus=="aerated":
            if data[i]["properties"]["time_series"][k]["status"]=="harvested":
                currentStatus = data[i]["properties"]["time_series"][k]["status"]
                statusList.append(data[i]["properties"]["time_series"][k]["status"])
                timeList.append(data[i]["properties"]["time_series"][k]["time"])
        elif currentStatus == "harvested":
            currentStatus = data[i]["properties"]["time_series"][k]["status"]
            statusList.append(data[i]["properties"]["time_series"][k]["status"])
            timeList.append(data[i]["properties"]["time_series"][k]["time"])
    timeseriesfeatures = []
    for k in range(0, len(timeList)):
        time_series_status = {}
        time_series_status["time"] = timeList[k]
        time_series_status["status"] = statusList[k]
        timeseriesfeatures.append(time_series_status) 
    data[i]["properties"]["time_series_status"] = timeseriesfeatures

In [15]:
print(unused)
print(active)
print(underpumping)
print(harvested)

2474
4006
335
973


In [16]:
with open("masterjson.json", "w+") as outfile:
    json.dump(data, outfile)

In [17]:
with open('./masterjson.json', 'r') as f:
  data = json.load(f)

In [18]:
dummyobject = {"type": "Feature",
               "properties": {}}

In [19]:
features = []
for i in range(0, len(data)):
    dummyobject = {"type": "Feature", "properties": {}}
    dummyobject["geometry"] = data[i]["polygon"]
    dummyobject["properties"] = data[i]["properties"]
    features.append(dummyobject)

In [20]:
objectjson = {"type": "FeatureCollection",
              "features": features}

In [21]:
with open("mastergeojson.json", "w+") as outfile:
    json.dump(objectjson, outfile)

### convert json to csv/excel

In [22]:
with open('./masterjson.json', 'r') as f:
  data = json.load(f)

In [23]:
daysOfCulture = []
for i in range(0, len(data)):
    timeseries = data[i]["properties"]["time_series"]
    if timeseries[len(timeSeries)-1]["status"]=="aerated":
        doc = 12
        for j in range(len(timeSeries)-2, 0, -1):
            if timeseries[j]["status"]!="aerated":
                continue
            else:
                doc = doc + 12
        doc = str(doc) + " to " + str(doc+12)
        daysOfCulture.append(doc)
    else:
        daysOfCulture.append("NA")

In [24]:
dataframe = gpd.read_file("./mastergeojson.json")

In [25]:
dataframe["Farmer Name"] = dataframe["farmer_name"]
dataframe["Village Name"] = dataframe["village_name"]
dataframe["Feed Supplier"] = dataframe["feed_supplier"]
dataframe["Pond Coordinates"] = dataframe["geometry"].centroid
dataframe["Farm Status"] = dataframe["farm_status"]
dataframe["Days of Culture"] = daysOfCulture

/var/folders/0n/fwtd0dwd2jd4tg26kf52g5kc0000gn/T/ipykernel_93946/4042232620.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dataframe["Pond Coordinates"] = dataframe["geometry"].centroid


In [26]:
mapslink = []
for i in range(0, len(dataframe)):
    c = str(dataframe["Pond Coordinates"][i])
    c = c.split(" ")
    c = "https://www.google.com/maps?q=" + c[2][:-1] + "%2C" + c[1][1:]
    mapslink.append(c)
dataframe["Pond Coordinates"] = mapslink

In [27]:
dataframe

,farmer_name,feed_supplier,village_name,id,name,time_series,farm_status,time_series_status,geometry,Farmer Name,Village Name,Feed Supplier,Pond Coordinates,Farm Status,Days of Culture
0,P PHANENDRA REDDY,SKRETTING,KOLANAKUDURU,1192,pond1192,"[ { ""time"": ""January-10-2023"", ""Water"": 1, ""Ae...",aerated,"[ { ""time"": ""January-22-2023"", ""status"": ""aera...","POLYGON ((79.94863 14.21654, 79.94867 14.21645...",P PHANENDRA REDDY,KOLANAKUDURU,SKRETTING,https://www.google.com/maps?q=14.2165435516199...,aerated,48 to 60
1,P PHANENDRA REDDY,SKRETTING,KOLANAKUDURU,1193,pond1193,"[ { ""time"": ""January-10-2023"", ""Water"": 1, ""Ae...",pumped,"[ { ""time"": ""January-22-2023"", ""status"": ""pump...","POLYGON ((79.94919 14.21757, 79.95032 14.21707...",P PHANENDRA REDDY,KOLANAKUDURU,SKRETTING,https://www.google.com/maps?q=14.2170760245521...,pumped,NA
2,M KOTESWARARAO,AVANTI,TIPPAGUNTAPALLEM,3021,pond3021,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((79.93757 14.14803, 79.93772 14.14852...",M KOTESWARARAO,TIPPAGUNTAPALLEM,AVANTI,https://www.google.com/maps?q=14.1481733618061...,empty,NA
3,M KOTESWARARAO,AVANTI,TIPPAGUNTAPALLEM,3022,pond3022,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",aerated,"[ { ""time"": ""April-16-2023"", ""status"": ""empty""...","POLYGON ((79.93813 14.14782, 79.93837 14.14827...",M KOTESWARARAO,TIPPAGUNTAPALLEM,AVANTI,https://www.google.com/maps?q=14.1479146060487...,aerated,60 to 72
4,V Venkaiah,Avanti,Kolanukudhuru,4,pond4,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",aerated,"[ { ""time"": ""April-04-2023"", ""status"": ""empty""...","POLYGON ((79.96092 14.20412, 79.96129 14.20402...",V Venkaiah,Kolanukudhuru,Avanti,https://www.google.com/maps?q=14.2043024003542...,aerated,60 to 72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7783,T PARAMDHAMA REDDY,NO CULTURE,PEDDAKATTAPALI,8046,pond8046,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((79.95681 14.15971, 79.95704 14.16045...",T PARAMDHAMA REDDY,PEDDAKATTAPALI,NO CULTURE,https://www.google.com/maps?q=14.1599831996315...,empty,NA
7784,RAJA REDDY,STOP CULTURE,PEDDAKATTAPALLI,8047,pond8047,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((79.95275 14.15435, 79.95226 14.15356...",RAJA REDDY,PEDDAKATTAPALLI,STOP CULTURE,https://www.google.com/maps?q=14.1537781731348...,empty,NA
7785,AVULA MANOHAR REDDY,AVANTI,PEDDAKATTAPALI,8048,pond8048,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",harvested,"[ { ""time"": ""April-16-2023"", ""status"": ""empty""...","POLYGON ((79.95882 14.15924, 79.95934 14.15892...",AVULA MANOHAR REDDY,PEDDAKATTAPALI,AVANTI,https://www.google.com/maps?q=14.1587204641326...,harvested,NA
7786,RADHA REDDY,CP,PEDDAKATTAPALI,8049,pond8049,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((79.96046 14.15810, 79.96105 14.15786...",RADHA REDDY,PEDDAKATTAPALI,CP,https://www.google.com/maps?q=14.1575589951511...,empty,NA


In [28]:
dataframe = dataframe.to_crs({'init': 'epsg:6933'})
dataframe["Area of Pond (hectare)"] = 100*dataframe["geometry"].area/10**6

/Users/azhanmohammed/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [29]:
dataframe

,farmer_name,feed_supplier,village_name,id,name,time_series,farm_status,time_series_status,geometry,Farmer Name,Village Name,Feed Supplier,Pond Coordinates,Farm Status,Days of Culture,Area of Pond (hectare)
0,P PHANENDRA REDDY,SKRETTING,KOLANAKUDURU,1192,pond1192,"[ { ""time"": ""January-10-2023"", ""Water"": 1, ""Ae...",aerated,"[ { ""time"": ""January-22-2023"", ""status"": ""aera...","POLYGON ((7713945.458 1795581.699, 7713950.115...",P PHANENDRA REDDY,KOLANAKUDURU,SKRETTING,https://www.google.com/maps?q=14.2165435516199...,aerated,48 to 60,1.016383
1,P PHANENDRA REDDY,SKRETTING,KOLANAKUDURU,1193,pond1193,"[ { ""time"": ""January-10-2023"", ""Water"": 1, ""Ae...",pumped,"[ { ""time"": ""January-22-2023"", ""status"": ""pump...","POLYGON ((7714000.308 1795709.060, 7714108.973...",P PHANENDRA REDDY,KOLANAKUDURU,SKRETTING,https://www.google.com/maps?q=14.2170760245521...,pumped,NA,0.815473
2,M KOTESWARARAO,AVANTI,TIPPAGUNTAPALLEM,3021,pond3021,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((7712878.733 1787100.812, 7712893.226...",M KOTESWARARAO,TIPPAGUNTAPALLEM,AVANTI,https://www.google.com/maps?q=14.1481733618061...,empty,NA,0.359555
3,M KOTESWARARAO,AVANTI,TIPPAGUNTAPALLEM,3022,pond3022,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",aerated,"[ { ""time"": ""April-16-2023"", ""status"": ""empty""...","POLYGON ((7712933.080 1787075.038, 7712955.855...",M KOTESWARARAO,TIPPAGUNTAPALLEM,AVANTI,https://www.google.com/maps?q=14.1479146060487...,aerated,60 to 72,0.417953
4,V Venkaiah,Avanti,Kolanukudhuru,4,pond4,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",aerated,"[ { ""time"": ""April-04-2023"", ""status"": ""empty""...","POLYGON ((7715131.472 1794044.706, 7715167.194...",V Venkaiah,Kolanukudhuru,Avanti,https://www.google.com/maps?q=14.2043024003542...,aerated,60 to 72,0.193663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7783,T PARAMDHAMA REDDY,NO CULTURE,PEDDAKATTAPALI,8046,pond8046,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((7714734.821 1788546.468, 7714757.595...",T PARAMDHAMA REDDY,PEDDAKATTAPALI,NO CULTURE,https://www.google.com/maps?q=14.1599831996315...,empty,NA,0.906015
7784,RAJA REDDY,STOP CULTURE,PEDDAKATTAPALLI,8047,pond8047,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((7714343.521 1787883.348, 7714296.420...",RAJA REDDY,PEDDAKATTAPALLI,STOP CULTURE,https://www.google.com/maps?q=14.1537781731348...,empty,NA,0.554237
7785,AVULA MANOHAR REDDY,AVANTI,PEDDAKATTAPALI,8048,pond8048,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",harvested,"[ { ""time"": ""April-16-2023"", ""status"": ""empty""...","POLYGON ((7714929.436 1788488.561, 7714979.642...",AVULA MANOHAR REDDY,PEDDAKATTAPALI,AVANTI,https://www.google.com/maps?q=14.1587204641326...,harvested,NA,0.620502
7786,RADHA REDDY,CP,PEDDAKATTAPALI,8049,pond8049,"[ { ""time"": ""January-10-2023"", ""Water"": 0, ""Ae...",empty,"[ { ""time"": ""June-15-2023"", ""status"": ""empty"" } ]","POLYGON ((7715087.301 1788348.154, 7715144.237...",RADHA REDDY,PEDDAKATTAPALI,CP,https://www.google.com/maps?q=14.1575589951511...,empty,NA,0.676199


In [30]:
dataframe = dataframe.drop(['farmer_name', 'feed_supplier', 'village_name', 'id', 'name', 'time_series_status', 'geometry', 'farm_status', 'time_series'], axis=1)

In [31]:
dataframe

,Farmer Name,Village Name,Feed Supplier,Pond Coordinates,Farm Status,Days of Culture,Area of Pond (hectare)
0,P PHANENDRA REDDY,KOLANAKUDURU,SKRETTING,https://www.google.com/maps?q=14.2165435516199...,aerated,48 to 60,1.016383
1,P PHANENDRA REDDY,KOLANAKUDURU,SKRETTING,https://www.google.com/maps?q=14.2170760245521...,pumped,NA,0.815473
2,M KOTESWARARAO,TIPPAGUNTAPALLEM,AVANTI,https://www.google.com/maps?q=14.1481733618061...,empty,NA,0.359555
3,M KOTESWARARAO,TIPPAGUNTAPALLEM,AVANTI,https://www.google.com/maps?q=14.1479146060487...,aerated,60 to 72,0.417953
4,V Venkaiah,Kolanukudhuru,Avanti,https://www.google.com/maps?q=14.2043024003542...,aerated,60 to 72,0.193663
...,...,...,...,...,...,...,...
7783,T PARAMDHAMA REDDY,PEDDAKATTAPALI,NO CULTURE,https://www.google.com/maps?q=14.1599831996315...,empty,NA,0.906015
7784,RAJA REDDY,PEDDAKATTAPALLI,STOP CULTURE,https://www.google.com/maps?q=14.1537781731348...,empty,NA,0.554237
7785,AVULA MANOHAR REDDY,PEDDAKATTAPALI,AVANTI,https://www.google.com/maps?q=14.1587204641326...,harvested,NA,0.620502
7786,RADHA REDDY,PEDDAKATTAPALI,CP,https://www.google.com/maps?q=14.1575589951511...,empty,NA,0.676199


In [ ]:
dataframe.to_csv("./analysis15June.csv", index=False)

In [ ]:
report = pd.read_excel("./Gudur Mapping Data.xlsx")

In [ ]:
report.columns

In [ ]:
len(report)

In [ ]:
report = report.dropna(subset=["Empty/Running"])

In [ ]:
len(report)

In [ ]:
report["Empty/Running"].unique()

In [ ]:
report["Ground Truth Status"] = "Running"
report.loc[report["Empty/Running"]=="Empty  ", "Ground Truth Status"] = "Empty"
report.loc[report["Empty/Running"]=="PUMPING", "Ground Truth Status"] = "Pumping"

In [ ]:
report

In [ ]:
report  = report.drop(['Farm Status', 'Days of Culture', 'Area of Pond (hectare)', 'Empty/Running'], axis=1)

In [ ]:
report

In [ ]:
report.to_csv("./filteredReport.csv")

In [ ]:
mergedRes = pd.merge(dataframe, report, on ='Pond Coordinates')

In [ ]:
mergedRes

In [ ]:
mergedRes.to_csv("crosscheck.csv", index=False)